# SwaptionPricing

In this notebook we illustrate European swaption pricing.

We use the QuantLib wrappers provided for the examples.

In [ ]:
import sys
sys.path.append('../') # make sure we can access the src/ folder

import matplotlib.pyplot as plt
import numpy as np

import QuantLib as ql
from src.yieldcurve import YieldCurve
from src.swaption import create_swaption


As preparation we first need to setup curves. We use the example curve data from earlier examples.

In [ ]:
terms = [    '1y',    '2y',    '3y',    '4y',    '5y',    '6y',    '7y',    '8y',    '9y',   '10y',   '12y',   '15y',   '20y',   '25y',   '30y', '50y'   ] 
rates = [ 2.70e-2, 2.75e-2, 2.80e-2, 3.00e-2, 3.36e-2, 3.68e-2, 3.97e-2, 4.24e-2, 4.50e-2, 4.75e-2, 4.75e-2, 4.70e-2, 4.50e-2, 4.30e-2, 4.30e-2, 4.30e-2 ] 
# rates = [ 0.025 for r in rates ]  # alternative flat curve example
rates2 = [ r+0.005 for r in rates ]
#
discCurve = YieldCurve(terms,rates)
projCurve = YieldCurve(terms,rates2)

In a first example we assume a flat volatility of 100bp.

In [ ]:
normal_implied_volatility = 100 * 1e-4

In our initial example we considered a Vanilla swap with 20y maturity and several *call options*. Each individual swap call option represents a European swaption on the *remaining* swap. As a consequence, the options start at different dates but the underlying swaps all mature in 20y.

A collection of swaptions with increasing expiry (and swap start) date and fixed swap maturity date is called the set of *co-terminal swaptions*. Note that the swap term (i.e. the swap end minus swap start) decreases for increasing expiry.

We construct the collection of co-terminal swaptions with first exercise in 1y and final maturity in 20y corresponding to our example Vanilla swap.

In [ ]:
maturity  = 20  # in years
swap_fixed_rate = 0.03

swaptions = []
labels = []
for k in range(1,maturity):
    expTerm = str(k)+'y'
    swpTerm = str(maturity-k)+'y'
    labels.append(expTerm+'-'+swpTerm)
    swaptions.append( create_swaption(expTerm,swpTerm, discCurve, projCurve, swap_fixed_rate, ql.VanillaSwap.Receiver, normal_implied_volatility ) )
    # The create_swaption wrapper constructs an instrument with 10,000 units notional.
    # Our swap example is supposed to have 100mm EUR notional, so we need to scale npv results by another 10,000


Now, we can compare the model prices of the co-terminal swaptions. 

In [ ]:
x = 4*np.linspace(1,19,19)
fig = plt.figure(figsize=(8, 4))
plt.bar(x,[ swaption.npv()*1e+4 for swaption in swaptions ],3.0)  # for 100mm notional
plt.xticks(x, labels, rotation='vertical')
plt.xlabel('co-terminal swaption')
plt.ylabel('option price')


For underlying swaps that are close to at-the-money we often see such a concave shape of co-terminal swaption prices.